In [1]:
import pyspark
from pyspark.sql.functions import *

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1638137095618_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# Read 
data = spark.read.option("delimiter", ",").csv("s3://data-apr/CPG0.csv", header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Data - Count of rows
data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

89770641

In [4]:
# Print Schema
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- customer_id: string (nullable = true)
 |-- retailer_channel: string (nullable = true)
 |-- purchase_time: string (nullable = true)
 |-- upc: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- price: string (nullable = true)
 |-- basket_total: string (nullable = true)
 |-- primary_category: string (nullable = true)
 |-- secondary_category: string (nullable = true)
 |-- purchased_zip: string (nullable = true)
 |-- registered_zip: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- receipt_item_id: string (nullable = true)
 |-- receipt_id: string (nullable = true)
 |-- identifier: string (nullable = true)
 |-- final_company: string (nullable = true)
 |-- brand: string (nullable = true)

In [5]:
# Show Data`
data.limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------+-------------------+--------------+--------------------+----------------+--------+-----+------------+--------------------+------------------+-------------+--------------+---+------+---------------+-----------+----------+--------------------+----------------+
|customer_id|retailer_channel|      purchase_time|           upc|        product_name|      brand_name|quantity|price|basket_total|    primary_category|secondary_category|purchased_zip|registered_zip|age|gender|receipt_item_id| receipt_id|identifier|       final_company|           brand|
+-----------+----------------+-------------------+--------------+--------------------+----------------+--------+-----+------------+--------------------+------------------+-------------+--------------+---+------+---------------+-----------+----------+--------------------+----------------+
|   10351180|         Grocery|2018-08-01 18:35:39|00000000040112|             BANANAS| Generic Produce|     2.0|0.605|       19.81|  

In [6]:
# Function - Split on Date
split_col = pyspark.sql.functions.split(data["purchase_time"], ' ')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Create additional columns
data = data.withColumn('date', split_col.getItem(0))
data = data.withColumn('time', split_col.getItem(1))
data = data.withColumn('day_of_week', dayofweek('date'))
data = data.withColumn('hour_of_day', hour('time'))
data = data.withColumn('day_of_week_string', pyspark.sql.functions.when(col('day_of_week') == 1, "Sunday")
                                                                  .when(col('day_of_week') == 2, "Monday")
                                                                  .when(col('day_of_week') == 3, "Tuesday")
                                                                  .when(col('day_of_week') == 4, "Wednesday")
                                                                  .when(col('day_of_week') == 5, "Thursday")
                                                                  .when(col('day_of_week') == 6, "Friday")
                                                                  .when(col('day_of_week') == 7, "Saturday")
                                                                  .otherwise("N/A"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
# Replace values in Gender column
data = data.na.fill("not specified", ["gender"])
data = data.replace('o', 'not specified', ['gender'])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# Column Null Values
data.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------+-------------+-------+------------+----------+--------+------+------------+----------------+------------------+-------------+--------------+---+------+---------------+----------+----------+-------------+--------+----+----+-----------+-----------+------------------+
|customer_id|retailer_channel|purchase_time|    upc|product_name|brand_name|quantity| price|basket_total|primary_category|secondary_category|purchased_zip|registered_zip|age|gender|receipt_item_id|receipt_id|identifier|final_company|   brand|date|time|day_of_week|hour_of_day|day_of_week_string|
+-----------+----------------+-------------+-------+------------+----------+--------+------+------------+----------------+------------------+-------------+--------------+---+------+---------------+----------+----------+-------------+--------+----+----+-----------+-----------+------------------+
|          0|               0|            0|2427364|     2452218|   1085537|  163978|311369|      893505|       

In [10]:
# Table 1a - Busy Times
table1 = data.groupBy("hour_of_day").count().orderBy(col("count").desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# Final Table - Table 1a
table1.createOrReplaceTempView("Table1")

spark.sql("CREATE TABLE table1 AS SELECT * FROM TABLE1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [12]:
# Table 2a - Popular Company (Listed)
table2 = data.groupBy("final_company").count().orderBy(col("count").desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Final Table - Table 2a
table2.createOrReplaceTempView("Table2")

spark.sql("CREATE TABLE table2 AS SELECT * FROM Table2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [14]:
# Table 3a - Popular Brand Name
table3 = data.groupBy("brand_name").count().orderBy(col("count").desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Final Table - Table 3a
table3.createOrReplaceTempView("Table3")

spark.sql("CREATE TABLE table3 AS SELECT * FROM Table3")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [16]:
# Table 4a - Popular Retail Channel
table4 = data.groupBy("retailer_channel").count().orderBy(col("count").desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Final Table - Table 4a
table4.createOrReplaceTempView("Table4")

spark.sql("CREATE TABLE table4 AS SELECT * FROM Table4")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [18]:
# Table 5a - Primary Category
table5 = data.groupBy("primary_category").count().orderBy(col("count").desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Final Table - Table 5a
table5.createOrReplaceTempView("Table5")

spark.sql("CREATE TABLE table5 AS SELECT * FROM Table5")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [20]:
# Table 6a - Gender
table6 = data.groupBy("gender").count().orderBy(col("count").desc())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Final Table - Table 6a
table6.createOrReplaceTempView("Table6")

spark.sql("CREATE TABLE table6 AS SELECT * FROM Table6")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]